In [1]:
from snowflake.snowpark import Session
import sys
import logging

# initiate logging at info level
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')

# snowpark session
def get_snowpark_session() -> Session:
    connection_parameters = {
       "ACCOUNT":"ZRGXHYE-TV29770",
        "USER":"snowpark_user",
        "PASSWORD":"Test@123",
        "ROLE":"SYSADMIN",
        "DATABASE":"SNOWFLAKE_SAMPLE_DATA",
        "SCHEMA":"TPCH_SF1",
        "WAREHOUSE":"SNOWSPARK_ETL"
    }
    # creating snowflake session object
    return Session.builder.configs(connection_parameters).create()   

def main():
    session = get_snowpark_session()

    context_df = session.sql("select current_role(), current_database(), current_schema(), current_warehouse()")
    context_df.show(3)

    customer_df = session.sql("select c_custkey,c_name,c_phone,c_mktsegment from snowflake_sample_data.tpch_sf1.customer limit 10")
    customer_df.show(5)

if __name__ == '__main__':
    main()  


03:09:26 - INFO - Snowflake Connector for Python Version: 3.1.0, Python Version: 3.9.13, Platform: Windows-10-10.0.22621
03:09:26 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
03:09:26 - INFO - Setting use_openssl_only mode to False
03:09:27 - INFO - Snowpark Session information: 
"version" : 1.6.1,
"python.version" : 3.9.13,
"python.connector.version" : 3.1.0,
"python.connector.session.id" : 635096814137434,
"os.name" : Windows

03:09:27 - INFO - query: [select current_role(), current_database(), current_schema(), current_warehouse()...]
03:09:27 - INFO - query execution done
03:09:27 - INFO - Number of results in first chunk: 0
03:09:27 - INFO - query: [SELECT  *  FROM (select current_role(), current_database(), current_schema(), cu...]
03:09:28 - INFO - query execution done
03:09:28 -

## Loading Data To Internal Stage Using Snowpark File API

In [28]:
import os
from snowflake.snowpark import Session
import sys
import logging

# initiate logging at info level
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')

# snowpark session
def get_snowpark_session() -> Session:
    connection_parameters = {
       "ACCOUNT":"ZRGXHYE-TV29770",
        "USER":"snowpark_user",
        "PASSWORD":"Test@123",
        "ROLE":"SYSADMIN",
        "DATABASE":"sales_dwh",
        "SCHEMA":"source"
    }
    # creating snowflake session object
    return Session.builder.configs(connection_parameters).create()   

def traverse_directory(directory,file_extension) -> list:
    local_file_path = []
    file_name = []  # List to store CSV file paths
    partition_dir = []
    print(directory)
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(file_extension):
                file_path = os.path.join(root, file)
                file_name.append(file)
                k=root.replace(directory, "")
                partition_dir.append(k.replace('\\','/'))
                local_file_path.append(file_path)

    return file_name,partition_dir,local_file_path

def main():
    # Specify the directory path to traverse
    directory_path = 'C:/Users/saura/Downloads/sales'
    csv_file_name, csv_partition_dir , csv_local_file_path= traverse_directory(directory_path,'.csv')
    parquet_file_name, parquet_partition_dir , parquet_local_file_path= traverse_directory(directory_path,'.parquet')
    json_file_name, json_partition_dir , json_local_file_path= traverse_directory(directory_path,'.json')
    stage_location = '@sales_dwh.source.my_internal_stg'
  
    
    csv_index = 0
    for file_element in csv_file_name:
        put_result = ( 
                    get_snowpark_session().file.put( 
                        csv_local_file_path[csv_index], 
                        "{0}/{1}".format(stage_location,csv_partition_dir[csv_index]), 
                        auto_compress=False, overwrite=True, parallel=10)
                    )
        #put_result(file_element," => ",put_result[0].status)
        csv_index+=1

    parquet_index = 0
    for file_element in parquet_file_name:

        put_result = ( 
                    get_snowpark_session().file.put( 
                        parquet_local_file_path[parquet_index], 
                        "{0}/{1}".format(stage_location,parquet_partition_dir[parquet_index]), 
                        auto_compress=False, overwrite=True, parallel=10)
                    )
        #put_result(file_element," => ",put_result[0].status)
        parquet_index+=1
    
    json_index = 0
    for file_element in parquet_file_name:

        put_result = ( 
                    get_snowpark_session().file.put( 
                        json_local_file_path[json_index], 
                        "{0}/{1}".format(stage_location,json_partition_dir[json_index]), 
                        auto_compress=False, overwrite=True, parallel=10)
                    )
        #put_result(file_element," => ",put_result[0].status)
        json_index+=1  
    
    
if __name__ == '__main__':
    main()

09:37:07 - INFO - closed
09:37:07 - INFO - closed
C:/Users/saura/Downloads/sales
C:/Users/saura/Downloads/sales
C:/Users/saura/Downloads/sales
		 ['order-20200101.csv', 'order-20200102.csv', 'order-20200103.csv', 'order-20200104.csv', 'order-20200105.csv', 'order-20200106.csv', 'order-20200107.csv', 'order-20200108.csv', 'order-20200109.csv', 'order-20200110.csv', 'order-20200111.csv', 'order-20200112.csv', 'order-20200113.csv', 'order-20200114.csv', 'order-20200115.csv', 'order-20200116.csv', 'order-20200117.csv', 'order-20200118.csv', 'order-20200119.csv', 'order-20200120.csv', 'order-20200121.csv', 'order-20200122.csv', 'order-20200123.csv', 'order-20200124.csv', 'order-20200125.csv', 'order-20200126.csv', 'order-20200127.csv', 'order-20200128.csv', 'order-20200129.csv', 'order-20200130.csv', 'order-20200131.csv'] 		 ['/source=IN/format=csv/date=2020-01-01', '/source=IN/format=csv/date=2020-01-02', '/source=IN/format=csv/date=2020-01-03', '/source=IN/format=csv/date=2020-01-04', '/s

IndexError: list index out of range

In [3]:
import os
from snowflake.snowpark import Session
import sys
import logging

# initiate logging at info level
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')

# snowpark session
def get_snowpark_session() -> Session:
    connection_parameters = {
       "ACCOUNT":"ZRGXHYE-TV29770",
        "USER":"snowpark_user",
        "PASSWORD":"Test@123",
        "ROLE":"SYSADMIN",
        "DATABASE":"sales_dwh",
        "SCHEMA":"source"
    }
    # creating snowflake session object
    return Session.builder.configs(connection_parameters).create()   

def main():
    get_snowpark_session().file.put("C:/Users/saura/Box/Data Engineer Project/Amazon Sales Analytics/exchange-rate-data.csv",
                                    "@sales_dwh.source.my_internal_stg/exchange/",auto_compress=False, overwrite=True, parallel=10)
    


if __name__ == '__main__':
    main()

05:11:39 - INFO - Snowflake Connector for Python Version: 3.1.0, Python Version: 3.9.13, Platform: Windows-10-10.0.22621
05:11:39 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
05:11:40 - INFO - Snowpark Session information: 
"version" : 1.6.1,
"python.version" : 3.9.13,
"python.connector.version" : 3.1.0,
"python.connector.session.id" : 635096814219270,
"os.name" : Windows

05:11:40 - INFO - query: [PUT 'file://C:/Users/saura/Box/Data Engineer Project/Amazon Sales Analytics/exch...]
05:11:40 - INFO - query execution done


## Moving Data from Stage To Source Schema - Snowpark API

In [11]:
import sys
import logging

from snowflake.snowpark import Session, DataFrame
from snowflake.snowpark.types import StructType, StringType, StructField, StringType,LongType,DecimalType,DateType,TimestampType
from snowflake.snowpark.functions import col,lit,row_number, rank
from snowflake.snowpark import Window

# initiate logging at info level
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')

# snowpark session
def get_snowpark_session() -> Session:
    connection_parameters = {
       "ACCOUNT":"ZRGXHYE-TV29770",
        "USER":"snowpark_user",
        "PASSWORD":"Test@123",
        "ROLE":"SYSADMIN",
        "DATABASE":"sales_dwh",
        "SCHEMA":"source",
        "warehouse":"snowspark_etl"
    }
    # creating snowflake session object
    return Session.builder.configs(connection_parameters).create()   

def ingest_in_sales(session)-> None:
    session.sql(" \
            copy into sales_dwh.source.in_sales_order from ( \
            select \
            sales_dwh.source.in_sales_order_seq.nextval, \
            t.$1::text as order_id, \
            t.$2::text as customer_name, \
            t.$3::text as mobile_key,\
            t.$4::number as order_quantity, \
            t.$5::number as unit_price, \
            t.$6::number as order_valaue,  \
            t.$7::text as promotion_code , \
            t.$8::number(10,2)  as final_order_amount,\
            t.$9::number(10,2) as tax_amount,\
            t.$10::date as order_dt,\
            t.$11::text as payment_status,\
            t.$12::text as shipping_status,\
            t.$13::text as payment_method,\
            t.$14::text as payment_provider,\
            t.$15::text as mobile,\
            t.$16::text as shipping_address,\
            metadata$filename as stg_file_name,\
            metadata$file_row_number as stg_row_numer,\
            metadata$file_last_modified as stg_last_modified\
            from \
            @sales_dwh.source.my_internal_stg/source=IN/format=csv/ \
            (                                                             \
                file_format => 'sales_dwh.common.my_csv_format'           \
            ) t  )  on_error = 'Continue'     \
            "
            ).collect()

def ingest_us_sales(session)-> None:
    session.sql(' \
            copy into sales_dwh.source.us_sales_order                \
            from                                    \
            (                                       \
                select                              \
                sales_dwh.source.us_sales_order_seq.nextval, \
                $1:"Order ID"::text as orde_id,   \
                $1:"Customer Name"::text as customer_name,\
                $1:"Mobile Model"::text as mobile_key,\
                to_number($1:"Quantity") as quantity,\
                to_number($1:"Price per Unit") as unit_price,\
                to_decimal($1:"Total Price") as total_price,\
                $1:"Promotion Code"::text as promotion_code,\
                $1:"Order Amount"::number(10,2) as order_amount,\
                to_decimal($1:"Tax") as tax,\
                $1:"Order Date"::date as order_dt,\
                $1:"Payment Status"::text as payment_status,\
                $1:"Shipping Status"::text as shipping_status,\
                $1:"Payment Method"::text as payment_method,\
                $1:"Payment Provider"::text as payment_provider,\
                $1:"Phone"::text as phone,\
                $1:"Delivery Address"::text as shipping_address,\
                metadata$filename as stg_file_name,\
                metadata$file_row_number as stg_row_numer,\
                metadata$file_last_modified as stg_last_modified\
                from                                \
                    @sales_dwh.source.my_internal_stg/source=US/format=parquet/\
                    (file_format => sales_dwh.common.my_parquet_format)\
                    ) on_error = continue \
            '
            ).collect()
    
def ingest_fr_sales(session)-> None:
    session.sql(' \
        copy into sales_dwh.source.fr_sales_order                                \
        from                                                    \
        (                                                       \
            select                                              \
            sales_dwh.source.fr_sales_order_seq.nextval,         \
            $1:"Order ID"::text as orde_id,                   \
            $1:"Customer Name"::text as customer_name,          \
            $1:"Mobile Model"::text as mobile_key,              \
            to_number($1:"Quantity") as quantity,               \
            to_number($1:"Price per Unit") as unit_price,       \
            to_decimal($1:"Total Price") as total_price,        \
            $1:"Promotion Code"::text as promotion_code,        \
            $1:"Order Amount"::number(10,2) as order_amount,    \
            to_decimal($1:"Tax") as tax,                        \
            $1:"Order Date"::date as order_dt,                  \
            $1:"Payment Status"::text as payment_status,        \
            $1:"Shipping Status"::text as shipping_status,      \
            $1:"Payment Method"::text as payment_method,        \
            $1:"Payment Provider"::text as payment_provider,    \
            $1:"Phone"::text as phone,                          \
            $1:"Delivery Address"::text as shipping_address ,    \
            metadata$filename as stg_file_name,\
            metadata$file_row_number as stg_row_numer,\
            metadata$file_last_modified as stg_last_modified\
            from                                                \
            @sales_dwh.source.my_internal_stg/source=FR/format=json/\
            (file_format => sales_dwh.common.my_json_format)\
            ) on_error=continue\
        '
        ).collect()

def main():

    #get the session object and get dataframe
    session = get_snowpark_session()

    print("<India Sales Order> Before")
    #session.sql("select count(*) from sales_dwh.source.in_sales_order").show()


    #ingest in sales data
    ingest_in_sales(session)

    session.sql("select count(*) from sales_dwh.source.in_sales_order").show()

    #ingest in sales data
    ingest_us_sales(session) 

    #ingest in sales data
    ingest_fr_sales(session)   

if __name__ == '__main__':
    main()

10:46:39 - INFO - Snowflake Connector for Python Version: 3.1.0, Python Version: 3.9.13, Platform: Windows-10-10.0.22621
10:46:39 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
10:46:39 - INFO - Snowpark Session information: 
"version" : 1.6.1,
"python.version" : 3.9.13,
"python.connector.version" : 3.1.0,
"python.connector.session.id" : 635096814215218,
"os.name" : Windows

<India Sales Order> Before
10:46:39 - INFO - query: [copy into sales_dwh.source.in_sales_order from (             select             ...]
10:46:42 - INFO - query execution done
10:46:42 - INFO - Number of results in first chunk: 31
10:46:42 - INFO - query: [select count(*) from sales_dwh.source.in_sales_order]
10:46:42 - INFO - query execution done
10:46:42 - INFO - Number of results in first chunk: 0
10:46:42 - INFO 

## Source to Curated (IN)

In [15]:
import sys
import logging

from snowflake.snowpark import Session, DataFrame
from snowflake.snowpark.functions import col,lit,row_number, rank
from snowflake.snowpark import Window

# initiate logging at info level
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')

# snowpark session
def get_snowpark_session() -> Session:
    connection_parameters = {
       "ACCOUNT":"ZRGXHYE-TV29770",
        "USER":"snowpark_user",
        "PASSWORD":"Test@123",
        "ROLE":"SYSADMIN",
        "DATABASE":"sales_dwh",
        "SCHEMA":"source",
         "warehouse":"snowspark_etl"
    }
    # creating snowflake session object
    return Session.builder.configs(connection_parameters).create()   

def filter_dataset(df, column_name, filter_criterian) -> DataFrame:
    # Payment Status = Paid
    # Shipping = Delivered
    return_df = df.filter(col(column_name) == filter_criterian)

    return return_df

def main():
    #get the session object and get dataframe
    session = get_snowpark_session()
    sales_df = session.sql("select * from sales_dwh.source.in_sales_order")

    # apply filter to select only paid and delivered sale orders
    # select * from in_sales_order where PAYMENT_STATUS = 'Paid' and SHIPPING_STATUS = 'Delivered'
    paid_sales_df = filter_dataset(sales_df,'PAYMENT_STATUS','Paid')
    shipped_sales_df = filter_dataset(paid_sales_df,'SHIPPING_STATUS','Delivered')

    # adding country and region to the data frame
    # select *, 'IN' as Country, 'APAC' as Region from in_sales_order where PAYMENT_STATUS = 'Paid' and SHIPPING_STATUS = 'Delivered'
    country_sales_df = shipped_sales_df.with_column('Country',lit('IN')).with_column('Region',lit('APAC'))

    # join to add forex calculation
    forex_df = session.sql("select * from sales_dwh.common.exchange_rate")
    sales_with_forext_df = country_sales_df.join(forex_df,country_sales_df['order_dt']==forex_df['date'],join_type='outer')
    #sales_with_forext_df.show(2)

    #de-duplication
    #print(sales_with_forext_df.count())
    unique_orders = sales_with_forext_df.with_column('order_rank',rank().over(Window.partitionBy(col("order_dt")).order_by(col('_metadata_last_modified').desc()))).filter(col("order_rank")==1).select(col('SALES_ORDER_KEY').alias('unique_sales_order_key'))
    final_sales_df = unique_orders.join(sales_with_forext_df,unique_orders['unique_sales_order_key']==sales_with_forext_df['SALES_ORDER_KEY'],join_type='inner')
    final_sales_df = final_sales_df.select(
        col('SALES_ORDER_KEY'),
        col('ORDER_ID'),
        col('ORDER_DT'),
        col('CUSTOMER_NAME'),
        col('MOBILE_KEY'),
        col('Country'),
        col('Region'),
        col('ORDER_QUANTITY'),
        lit('INR').alias('LOCAL_CURRENCY'),
        col('UNIT_PRICE').alias('LOCAL_UNIT_PRICE'),
        col('PROMOTION_CODE').alias('PROMOTION_CODE'),
        col('FINAL_ORDER_AMOUNT').alias('LOCAL_TOTAL_ORDER_AMT'),
        col('TAX_AMOUNT').alias('local_tax_amt'),
        col('USD2INR').alias("Exhchange_Rate"),
        (col('FINAL_ORDER_AMOUNT')/col('USD2INR')).alias('US_TOTAL_ORDER_AMT'),
        (col('TAX_AMOUNT')/col('USD2INR')).alias('USD_TAX_AMT'),
        col('payment_status'),
        col('shipping_status'),
        col('payment_method'),
        col('payment_provider'),
        col('mobile').alias('conctact_no'),
        col('shipping_address')
    )

    #final_sales_df.show(5)
    final_sales_df.write.save_as_table("sales_dwh.curated.in_sales_order",mode="append")
    
if __name__ == '__main__':
    main()

10:50:56 - INFO - Snowflake Connector for Python Version: 3.1.0, Python Version: 3.9.13, Platform: Windows-10-10.0.22621
10:50:56 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.


10:50:57 - INFO - Snowpark Session information: 
"version" : 1.6.1,
"python.version" : 3.9.13,
"python.connector.version" : 3.1.0,
"python.connector.session.id" : 635096814227478,
"os.name" : Windows

10:50:57 - INFO - query: [select * from sales_dwh.source.in_sales_order]
10:50:58 - INFO - query execution done
10:50:58 - INFO - Number of results in first chunk: 0
10:50:58 - INFO - query: [select * from sales_dwh.common.exchange_rate]
10:50:58 - INFO - query execution done
10:50:58 - INFO - Number of results in first chunk: 0
10:50:58 - INFO - query: [SELECT "SALES_ORDER_KEY" AS "SALES_ORDER_KEY", "ORDER_ID" AS "ORDER_ID", "CUSTOM...]
10:50:58 - INFO - query execution done
10:50:58 - INFO - Number of results in first chunk: 0
10:50:58 - INFO - query: [SELECT "DATE" AS "DATE", "USD2USD" AS "USD2USD", "USD2EU" AS "USD2EU", "USD2CAN"...]
10:50:59 - INFO - query execution done
10:50:59 - INFO - Number of results in first chunk: 0
10:50:59 - INFO - query: [SELECT  *  FROM (( SELECT NULL :: 

## Source to Curated (US)

In [16]:
import sys
import logging

from snowflake.snowpark import Session, DataFrame
from snowflake.snowpark.functions import col,lit,row_number, rank
from snowflake.snowpark import Window

# initiate logging at info level
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')

# snowpark session
def get_snowpark_session() -> Session:
    connection_parameters = {
       "ACCOUNT":"ZRGXHYE-TV29770",
        "USER":"snowpark_user",
        "PASSWORD":"Test@123",
        "ROLE":"SYSADMIN",
        "DATABASE":"sales_dwh",
        "SCHEMA":"source",
         "warehouse":"snowspark_etl"
    }
    # creating snowflake session object
    return Session.builder.configs(connection_parameters).create()   

def filter_dataset(df, column_name, filter_criterian) -> DataFrame:
    # Payment Status = Paid
    # Shipping = Delivered
    return_df = df.filter(col(column_name) == filter_criterian)

    return return_df

def main():

    #get the session object and get dataframe
    session = get_snowpark_session()
    sales_df = session.sql("select * from us_sales_order")

    # apply filter to select only paid and delivered sale orders
    # select * from us_sales_order where PAYMENT_STATUS = 'Paid' and SHIPPING_STATUS = 'Delivered'
    paid_sales_df = filter_dataset(sales_df,'PAYMENT_STATUS','Paid')
    shipped_sales_df = filter_dataset(paid_sales_df,'SHIPPING_STATUS','Delivered')

    # adding country and region to the data frame
    # select *, 'IN' as Country, 'APAC' as Region from us_sales_order where PAYMENT_STATUS = 'Paid' and SHIPPING_STATUS = 'Delivered'
    country_sales_df = shipped_sales_df.with_column('Country',lit('US')).with_column('Region',lit('NA'))

    # join to add forex calculation
    forex_df = session.sql("select * from sales_dwh.common.exchange_rate")
    sales_with_forext_df = country_sales_df.join(forex_df,country_sales_df['order_dt']==forex_df['date'],join_type='outer')
    #sales_with_forext_df.show(2)

    #de-duplication
    print(sales_with_forext_df.count())
    unique_orders = sales_with_forext_df.with_column('order_rank',rank().over(Window.partitionBy(col("order_dt")).order_by(col('_metadata_last_modified').desc()))).filter(col("order_rank")==1).select(col('SALES_ORDER_KEY').alias('unique_sales_order_key'))
    final_sales_df = unique_orders.join(sales_with_forext_df,unique_orders['unique_sales_order_key']==sales_with_forext_df['SALES_ORDER_KEY'],join_type='inner')
    final_sales_df = final_sales_df.select(
        col('SALES_ORDER_KEY'),
        col('ORDER_ID'),
        col('ORDER_DT'),
        col('CUSTOMER_NAME'),
        col('MOBILE_KEY'),
         col('Country'),
        col('Region'),
        col('ORDER_QUANTITY'),
        lit('USD').alias('LOCAL_CURRENCY'),
        col('UNIT_PRICE').alias('LOCAL_UNIT_PRICE'),
        col('PROMOTION_CODE').alias('PROMOTION_CODE'),
        col('FINAL_ORDER_AMOUNT').alias('LOCAL_TOTAL_ORDER_AMT'),
        col('TAX_AMOUNT').alias('local_tax_amt'),
        col('USD2INR').alias("Exhchange_Rate"),
        (col('FINAL_ORDER_AMOUNT')/col('USD2USD')).alias('US_TOTAL_ORDER_AMT'),
        (col('TAX_AMOUNT')/col('USD2USD')).alias('USD_TAX_AMT'),
        col('payment_status'),
        col('shipping_status'),
        col('payment_method'),
        col('payment_provider'),
        col('phone').alias('conctact_no'),
        col('shipping_address')
    )

    #final_sales_df.show(5)
    final_sales_df.write.save_as_table("sales_dwh.curated.us_sales_order",mode="append")
    
if __name__ == '__main__':
    main()

11:04:43 - INFO - Snowflake Connector for Python Version: 3.1.0, Python Version: 3.9.13, Platform: Windows-10-10.0.22621
11:04:43 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
11:04:43 - INFO - Snowpark Session information: 
"version" : 1.6.1,
"python.version" : 3.9.13,
"python.connector.version" : 3.1.0,
"python.connector.session.id" : 635096814223370,
"os.name" : Windows

11:04:43 - INFO - query: [select * from us_sales_order]
11:04:44 - INFO - query execution done
11:04:44 - INFO - Number of results in first chunk: 0
11:04:44 - INFO - query: [select * from sales_dwh.common.exchange_rate]
11:04:44 - INFO - query execution done
11:04:44 - INFO - Number of results in first chunk: 0
11:04:44 - INFO - query: [SELECT "SALES_ORDER_KEY" AS "SALES_ORDER_KEY", "ORDER_ID" AS "ORDER_ID", "CUSTOM.

## Source to Curated (FR)

In [21]:
import sys
import logging

from snowflake.snowpark import Session, DataFrame
from snowflake.snowpark.functions import col,lit,row_number, rank
from snowflake.snowpark import Window

# initiate logging at info level
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')

# snowpark session
def get_snowpark_session() -> Session:
    connection_parameters = {
       "ACCOUNT":"ZRGXHYE-TV29770",
        "USER":"snowpark_user",
        "PASSWORD":"Test@123",
        "ROLE":"SYSADMIN",
        "DATABASE":"sales_dwh",
        "SCHEMA":"source",
         "warehouse":"snowspark_etl"
    }
    # creating snowflake session object
    return Session.builder.configs(connection_parameters).create()   

def filter_dataset(df, column_name, filter_criterian) -> DataFrame:
    # Payment Status = Paid
    # Shipping = Delivered
    return_df = df.filter(col(column_name) == filter_criterian)

    return return_df

def main():

    #get the session object and get dataframe
    session = get_snowpark_session()
    sales_df = session.sql("select * from us_sales_order")

    # apply filter to select only paid and delivered sale orders
    # select * from us_sales_order where PAYMENT_STATUS = 'Paid' and SHIPPING_STATUS = 'Delivered'
    paid_sales_df = filter_dataset(sales_df,'PAYMENT_STATUS','Paid')
    shipped_sales_df = filter_dataset(paid_sales_df,'SHIPPING_STATUS','Delivered')

    # adding country and region to the data frame
    # select *, 'IN' as Country, 'APAC' as Region from us_sales_order where PAYMENT_STATUS = 'Paid' and SHIPPING_STATUS = 'Delivered'
    country_sales_df = shipped_sales_df.with_column('Country',lit('FR')).with_column('Region',lit('EU'))
    
    # join to add forex calculation
    forex_df = session.sql("select * from sales_dwh.common.exchange_rate")
    sales_with_forext_df = country_sales_df.join(forex_df,country_sales_df['order_dt']==forex_df['date'],join_type='outer')
    #sales_with_forext_df.show(2)

    #de-duplication
    print(sales_with_forext_df.count())
    unique_orders = sales_with_forext_df.with_column('order_rank',rank().over(Window.partitionBy(col("order_dt")).order_by(col('_metadata_last_modified').desc()))).filter(col("order_rank")==1).select(col('SALES_ORDER_KEY').alias('unique_sales_order_key'))
    final_sales_df = unique_orders.join(sales_with_forext_df,unique_orders['unique_sales_order_key']==sales_with_forext_df['SALES_ORDER_KEY'],join_type='inner')
    final_sales_df = final_sales_df.select(
        col('SALES_ORDER_KEY'),
        col('ORDER_ID'),
        col('ORDER_DT'),
        col('CUSTOMER_NAME'),
        col('MOBILE_KEY'),
        col('Country'),
        col('Region'),
        col('ORDER_QUANTITY'),
        lit('EUR').alias('LOCAL_CURRENCY'),
        col('UNIT_PRICE').alias('LOCAL_UNIT_PRICE'),
        col('PROMOTION_CODE').alias('PROMOTION_CODE'),
        col('FINAL_ORDER_AMOUNT').alias('LOCAL_TOTAL_ORDER_AMT'),
        col('TAX_AMOUNT').alias('local_tax_amt'),
        col('USD2EU').alias("Exhchange_Rate"),
        (col('FINAL_ORDER_AMOUNT')/col('USD2EU')).alias('US_TOTAL_ORDER_AMT'),
        (col('TAX_AMOUNT')/col('USD2EU')).alias('USD_TAX_AMT'),
        col('payment_status'),
        col('shipping_status'),
        col('payment_method'),
        col('payment_provider'),
        col('phone').alias('conctact_no'),
        col('shipping_address')
    )

    #final_sales_df.show(5)
    final_sales_df.write.save_as_table("sales_dwh.curated.fr_sales_order",mode="append")
    
if __name__ == '__main__':
    main()

11:33:03 - INFO - Snowflake Connector for Python Version: 3.1.0, Python Version: 3.9.13, Platform: Windows-10-10.0.22621
11:33:03 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
11:33:03 - INFO - Snowpark Session information: 
"version" : 1.6.1,
"python.version" : 3.9.13,
"python.connector.version" : 3.1.0,
"python.connector.session.id" : 635096814239754,
"os.name" : Windows

11:33:03 - INFO - query: [select * from us_sales_order]
11:33:03 - INFO - query execution done
11:33:03 - INFO - Number of results in first chunk: 0
11:33:03 - INFO - query: [select * from sales_dwh.common.exchange_rate]
11:33:04 - INFO - query execution done
11:33:04 - INFO - Number of results in first chunk: 0
11:33:04 - INFO - query: [SELECT "SALES_ORDER_KEY" AS "SALES_ORDER_KEY", "ORDER_ID" AS "ORDER_ID", "CUSTOM.

##  Curated To Model Snowpark Python Example

In [ ]:
import sys
import logging
import pandas as pd

from snowflake.snowpark import Session, DataFrame, CaseExpr
from snowflake.snowpark.functions import col,lit,row_number, rank, split,cast, when, expr,min, max
from snowflake.snowpark.types import StructType, StringType, StructField, StringType,LongType,DecimalType,DateType,TimestampType
from snowflake.snowpark import Window

# initiate logging at info level
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')

# snowpark session
def get_snowpark_session() -> Session:
    connection_parameters = {
       "ACCOUNT":"abc1234",
        "USER":"demo",
        "PASSWORD":"simplepwd",
        "ROLE":"SYSADMIN",
        "DATABASE":"sales_dwh",
        "SCHEMA":"source"
    }
    # creating snowflake session object
    return Session.builder.configs(connection_parameters).create()   

# This is a simple dim table having nation and region.
# fields are 'Country','Region'
def create_region_dim(all_sales_df,session)-> None:
    region_dim_df = all_sales_df.groupBy(col("Country"),col("Region")).count()
    region_dim_df.show(2)
    region_dim_df = region_dim_df.with_column("isActive",lit('Y'))
    region_dim_df = region_dim_df.selectExpr("sales_dwh.source.region_dim_seq.nextval as region_id_pk","Country", "Region", "isActive") 
    #region_dim_df.write.save_as_table('sales_dwh.consumption.region_dim',mode="append")   

    region_dim_df.show(5)
    # part 2 where delta data will be processed 
    
    existing_region_dim_df = session.sql("select Country, Region from sales_dwh.consumption.region_dim")

    region_dim_df = region_dim_df.join(existing_region_dim_df,region_dim_df['Country']==existing_region_dim_df['Country'],join_type='leftanti')
    region_dim_df.show(5)
    intsert_cnt = int(region_dim_df.count())
    if intsert_cnt>0:
        region_dim_df.write.save_as_table("sales_dwh.consumption.region_dim",mode="append")
        print("save operation ran...")
    else:
        print("No insert ...Opps...")


    # have exclude key


def create_product_dim(all_sales_df,session)-> None:

    product_dim_df = all_sales_df.with_column("Brand",split(col('MOBILE_KEY'),lit('/'))[0])     \
                                .with_column("Model",split(col('MOBILE_KEY'),lit('/'))[1])      \
                                .with_column("Color",split(col('MOBILE_KEY'),lit('/'))[2])      \
                                .with_column("Memory",split(col('MOBILE_KEY'),lit('/'))[3])     \
                                .select(col('mobile_key'),col('Brand'),col('Model'),col('Color'),col('Memory'))
    
    product_dim_df = product_dim_df.select(col('mobile_key'),                    \
                                        cast(col('Brand'), StringType()).as_("Brand"),\
                                        cast(col('Model'), StringType()).as_("Model"),\
                                        cast(col('Color'), StringType()).as_("Color"),\
                                        cast(col('Memory'), StringType()).as_("Memory")\
                                        )
    
    product_dim_df = product_dim_df.groupBy(col('mobile_key'),col("Brand"),col("Model"),col("Color"),col("Memory")).count()
    product_dim_df = product_dim_df.with_column("isActive",lit('Y'))

    #fetch existing product dim records.
    existing_product_dim_df = session.sql("select mobile_key, Brand, Model, Color, Memory from sales_dwh.consumption.product_dim")
    existing_product_dim_df.count()

    product_dim_df = product_dim_df.join(existing_product_dim_df,["mobile_key", "Brand", "Model", "Color", "Memory"],join_type='leftanti')

    product_dim_df.show(5)

    product_dim_df = product_dim_df.selectExpr("sales_dwh.consumption.product_dim_seq.nextval as product_id_pk","mobile_key","Brand", "Model","Color","Memory", "isActive") 

    product_dim_df.show(5)
    intsert_cnt = int(product_dim_df.count())
    if intsert_cnt>0:
        product_dim_df.write.save_as_table("sales_dwh.consumption.product_dim",mode="append")
        print("save operation ran...")
    else:
        print("No insert ...Opps...")

def create_promocode_dim(all_sales_df,session)-> None:


    promo_code_dim_df = all_sales_df.with_column( "promotion_code", expr("case when promotion_code is null then 'NA' else promotion_code end"))
    promo_code_dim_df = promo_code_dim_df.groupBy(col("promotion_code"),col("country"),col("region")).count()
    promo_code_dim_df = promo_code_dim_df.with_column("isActive",lit('Y'))

    #promo_code_dim_df.show(10)

    
    #fetch existing product dim records.
    existing_promo_code_dim_df = session.sql("select promotion_code, country, region from sales_dwh.consumption.promo_code_dim")


    promo_code_dim_df = promo_code_dim_df.join(existing_promo_code_dim_df,["promotion_code", "country", "region"],join_type='leftanti')


    promo_code_dim_df = promo_code_dim_df.selectExpr("sales_dwh.consumption.promo_code_dim_seq.nextval as promo_code_id_pk","promotion_code", "country","region","isActive") 


    intsert_cnt = int(promo_code_dim_df.count())
    if intsert_cnt>0:
        promo_code_dim_df.write.save_as_table("sales_dwh.consumption.promo_code_dim",mode="append")
        print("save operation ran...")
    else:
        print("No insert ...Opps...")
    
def create_customer_dim(all_sales_df, session) -> None:
    customer_dim_df = all_sales_df.groupBy(col("COUNTRY"),col("REGION"),col("CUSTOMER_NAME"),col("CONCTACT_NO"),col("SHIPPING_ADDRESS")).count()
    customer_dim_df = customer_dim_df.with_column("isActive",lit('Y'))
    customer_dim_df = customer_dim_df.selectExpr("customer_name", "conctact_no","shipping_address","country","region" ,"isactive") 
    #region_dim_df.write.save_as_table('sales_dwh.consumption.region_dim',mode="append")   

    customer_dim_df.show(5)
    # part 2 where delta data will be processed 
    
    existing_customer_dim_df = session.sql("select customer_name,conctact_no,shipping_address,country, region from sales_dwh.consumption.customer_dim")

    customer_dim_df = customer_dim_df.join(existing_customer_dim_df,["customer_name","conctact_no","shipping_address","country", "region"],join_type='leftanti')

    customer_dim_df = customer_dim_df.selectExpr("sales_dwh.consumption.customer_dim_seq.nextval as customer_id_pk","customer_name", "conctact_no","shipping_address","country","region", "isActive") 

    customer_dim_df.show(5)

    intsert_cnt = int(customer_dim_df.count())
    if intsert_cnt>0:
        customer_dim_df.write.save_as_table("sales_dwh.consumption.customer_dim",mode="append")
        print("save operation ran...")
    else:
        print("No insert ...Opps...")
    
def create_payment_dim(all_sales_df, session) -> None:
    payment_dim_df = all_sales_df.groupBy(col("COUNTRY"),col("REGION"),col("payment_method"),col("payment_provider")).count()
    payment_dim_df = payment_dim_df.with_column("isActive",lit('Y'))

    #region_dim_df.write.save_as_table('sales_dwh.consumption.region_dim',mode="append")   

    payment_dim_df.show(5)
    # part 2 where delta data will be processed 
    
    existing_payment_dim_df = session.sql("select payment_method,payment_provider,country, region from sales_dwh.consumption.payment_dim")

    payment_dim_df = payment_dim_df.join(existing_payment_dim_df,["payment_method","payment_provider","country", "region"],join_type='leftanti')

    payment_dim_df = payment_dim_df.selectExpr("sales_dwh.consumption.payment_dim_seq.nextval as payment_id_pk","payment_method", "payment_provider","country","region", "isActive") 


    intsert_cnt = int(payment_dim_df.count())
    if intsert_cnt>0:
        payment_dim_df.write.save_as_table("sales_dwh.consumption.payment_dim",mode="append")
        print("save operation ran...")
    else:
        print("No insert ...Opps...")

def create_date_dim(all_sales_df, session) -> None:
    start_date = all_sales_df.select(min("order_dt").alias("min_order_dt")).collect()[0].as_dict()['MIN_ORDER_DT']
    end_date = all_sales_df.select(max("order_dt").alias("max_order_dt")).collect()[0].as_dict()['MAX_ORDER_DT']
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    #print(date_range)
    date_dim = pd.DataFrame()
    date_dim['order_dt'] = date_range.date
    date_dim['Year'] = date_range.year
    # Calculate day counter
    start_day_of_year = pd.to_datetime(start_date).dayofyear
    date_dim['DayCounter'] = date_range.dayofyear - start_day_of_year + 1

    date_dim['Month'] = date_range.month
    date_dim['Quarter'] = date_range.quarter
    date_dim['Day'] = date_range.day
    date_dim['DayOfWeek'] = date_range.dayofweek
    date_dim['DayName'] = date_range.strftime('%A')
    date_dim['DayOfMonth'] = date_range.day
    date_dim['Weekday'] = date_dim['DayOfWeek'].map({0: 'Weekday', 1: 'Weekday', 2: 'Weekday', 3: 'Weekday', 4: 'Weekday', 5: 'Weekend', 6: 'Weekend'})


    date_dim_df = session.create_dataframe(date_dim)

    existing_date_dim_df = session.sql("select order_dt from sales_dwh.consumption.date_dim ") 
    date_dim_df = date_dim_df.join(existing_date_dim_df,existing_date_dim_df['order_dt']==date_dim_df['"order_dt"'],join_type='leftanti')

    date_dim_df = date_dim_df.selectExpr(' \
                                   sales_dwh.consumption.date_dim_seq.nextval, \
                                   "order_dt" as order_dt, \
                                   "DayCounter" as day_counter,\
                                   "Year" as order_year, \
                                   "Month" as order_month, \
                                   "Quarter" as order_quarter, \
                                   "Day" as order_day, \
                                   "DayOfWeek" as order_dayofweek, \
                                   "DayName" as order_dayname, \
                                   "DayOfMonth" as order_dayofmonth, \
                                   "Weekday" as order_weekday\
                                   ')

    intsert_cnt = int(date_dim_df.count())
    if intsert_cnt>0:
        date_dim_df.write.save_as_table("sales_dwh.consumption.date_dim",mode="append")
        print("save operation ran...")
    else:
        print("No insert ...Opps...")

def main():
    #get the session object and get dataframe
    session = get_snowpark_session()

    in_sales_df = session.sql("select * from sales_dwh.curated.in_sales_order")
    us_sales_df = session.sql("select * from sales_dwh.curated.us_sales_order")
    fr_sales_df = session.sql("select * from sales_dwh.curated.fr_sales_order")

    all_sales_df = in_sales_df.union(us_sales_df).union(fr_sales_df)

    create_date_dim(all_sales_df,session)       #date dimension


    create_region_dim(all_sales_df,session)     #region dimension
    create_product_dim(all_sales_df,session)    #product dimension
    create_promocode_dim(all_sales_df,session)  #promot code dimension
    create_customer_dim(all_sales_df,session)   #customer dimension
    create_payment_dim(all_sales_df,session)    #payment dimension
    create_date_dim(all_sales_df,session)       #date dimension


    date_dim_df = session.sql("select date_id_pk, order_dt from sales_dwh.consumption.date_dim")
    customer_dim_df = session.sql("select customer_id_pk, customer_name, country, region from sales_dwh.consumption.CUSTOMER_DIM")
    payment_dim_df = session.sql("select payment_id_pk, payment_method, payment_provider, country, region from sales_dwh.consumption.PAYMENT_DIM")
    product_dim_df = session.sql("select product_id_pk, mobile_key from sales_dwh.consumption.PRODUCT_DIM")
    promo_code_dim_df = session.sql("select promo_code_id_pk,promotion_code,country, region from sales_dwh.consumption.PROMO_CODE_DIM")
    region_dim_df = session.sql("select region_id_pk,country, region from sales_dwh.consumption.REGION_DIM")

    all_sales_df = all_sales_df.with_column( "promotion_code", expr("case when promotion_code is null then 'NA' else promotion_code end"))
    all_sales_df = all_sales_df.join(date_dim_df, ["order_dt"],join_type='inner')
    all_sales_df = all_sales_df.join(customer_dim_df, ["customer_name","region","country"],join_type='inner')
    all_sales_df = all_sales_df.join(payment_dim_df, ["payment_method", "payment_provider", "country", "region"],join_type='inner')
    #all_sales_df = all_sales_df.join(product_dim_df, ["brand","model","color","Memory"],join_type='inner')
    all_sales_df = all_sales_df.join(product_dim_df, ["mobile_key"],join_type='inner')
    all_sales_df = all_sales_df.join(promo_code_dim_df, ["promotion_code","country", "region"],join_type='inner')
    all_sales_df = all_sales_df.join(region_dim_df, ["country", "region"],join_type='inner')
    all_sales_df = all_sales_df.selectExpr("sales_dwh.consumption.sales_fact_seq.nextval as order_id_pk, \
                                           order_id as order_code,                               \
                                           date_id_pk as date_id_fk,          \
                                           region_id_pk as region_id_fk,            \
                                           customer_id_pk as customer_id_fk,        \
                                           payment_id_pk as payment_id_fk,          \
                                           product_id_pk as product_id_fk,          \
                                           promo_code_id_pk as promo_code_id_fk,    \
                                           order_quantity,                          \
                                           local_total_order_amt,                   \
                                           local_tax_amt,                           \
                                           exhchange_rate,                          \
                                           us_total_order_amt,                      \
                                           usd_tax_amt                              \
                                           ")
    all_sales_df.write.save_as_table("sales_dwh.consumption.sales_fact",mode="append")


if __name__ == '__main__':
    main()

In [8]:
!pip install pyarrow==10.0.1